In [98]:
import flair
import pandas as pd
import torch as pt
import requests
import matplotlib as plt
from datetime import datetime, timedelta

In [117]:
sentiment_model = flair.models.TextClassifier.load('en-sentiment')

bearer_token = "AAAAAAAAAAAAAAAAAAAAANUsKQEAAAAA0UrO%2Fzamc57%2FS1xAnkdNjJwl3bw%3DNFWcIjGzshLLZpAUksltVEfg2WWrG9DP3EYunyWSmrrHW3GSte"
params = {
    'q': '(covid OR covid-19 OR coronavirus OR rona)(lang:en)',
    'tweet_mode': 'extended',
    'result_type': 'popular',
    'lang': 'en',
    'count': '1'
}



dtformat = '%Y-%m-%dT%H:%M:%SZ'
now = datetime.now()
now = now.strftime(dtformat)

def time_travel(now, days):
    now = datetime.strptime(now, dtformat)
    back_in_time = now - timedelta(days=days)
    return back_in_time.strftime(dtformat)

def time_travelMins(now, mins):
    now = datetime.strptime(now, dtformat)
    back_in_time = now - timedelta(minutes=mins)
    return back_in_time.strftime(dtformat)

def get_data(tweet):
    data = {
        'id': tweet['id_str'],
        'created_at': tweet['created_at'],
        'text': tweet['full_text']
    }
    return data

df = pd.DataFrame()

targetDate = time_travel(now, 521)
currentDate = now

while True:
    if currentDate < targetDate:
        print("Done")
        break
    pre60 = time_travelMins(currentDate, 60)
    params['start_time'] = pre60
    params['end_time'] = currentDate
    response = requests.get(
    'https://api.twitter.com/1.1/search/tweets.json?',
    params=params,
    headers={
        'authorization': 'Bearer '+ bearer_token
    })
    for tweet in response.json()['statuses']:
        row = get_data(tweet)
        df = df.append(row, ignore_index=True)        
    
    currentDate = time_travel(currentDate, 7)
    print("DATE: ", currentDate)

df.head()

2021-06-29 14:59:42,347 loading file /Users/saadimran/.flair/models/sentiment-en-mix-distillbert_4.pt
DATE:  2021-06-22T14:59:50Z
DATE:  2021-06-15T14:59:50Z
DATE:  2021-06-08T14:59:50Z
DATE:  2021-06-01T14:59:50Z
DATE:  2021-05-25T14:59:50Z
DATE:  2021-05-18T14:59:50Z
DATE:  2021-05-11T14:59:50Z
DATE:  2021-05-04T14:59:50Z
DATE:  2021-04-27T14:59:50Z
DATE:  2021-04-20T14:59:50Z
DATE:  2021-04-13T14:59:50Z
DATE:  2021-04-06T14:59:50Z
DATE:  2021-03-30T14:59:50Z
DATE:  2021-03-23T14:59:50Z
DATE:  2021-03-16T14:59:50Z
DATE:  2021-03-09T14:59:50Z
DATE:  2021-03-02T14:59:50Z
DATE:  2021-02-23T14:59:50Z
DATE:  2021-02-16T14:59:50Z
DATE:  2021-02-09T14:59:50Z
DATE:  2021-02-02T14:59:50Z
DATE:  2021-01-26T14:59:50Z
DATE:  2021-01-19T14:59:50Z
DATE:  2021-01-12T14:59:50Z
DATE:  2021-01-05T14:59:50Z
DATE:  2020-12-29T14:59:50Z
DATE:  2020-12-22T14:59:50Z
DATE:  2020-12-15T14:59:50Z
DATE:  2020-12-08T14:59:50Z
DATE:  2020-12-01T14:59:50Z
DATE:  2020-11-24T14:59:50Z
DATE:  2020-11-17T14:59:50Z
DA

""


In [81]:
# Clean the text in all tweets
def cleanText(row):
    whitespace = re.compile(r"\s+")
    web_address = re.compile(r"(?i)http(s):\/\/[a-z0-9.~_\-\/]+")
    tesla = re.compile(r"(?i)@Tesla(?=\b)")
    user = re.compile(r"(?i)@[a-z0-9_]+")
    
    text = row["text"]
    text = whitespace.sub(' ', text)
    text = web_address.sub('', text)
    text = tesla.sub('Covid', text)
    text = user.sub('', text)
    return text

df["cleanText"] = df.apply(lambda row: cleanText(row), axis = 1)

In [82]:
df.loc[1]["text"]

'RT @CUP_PoliSci: #FirstView from @PoPpublicsphere -\n\nCOVID-19 and the Paradox of Scientific Advice - https://t.co/UEuJ1VZGVI\n\n- @zpamuk \n\n#…'

In [83]:
df.loc[1]["cleanText"]

'RT : #FirstView from  - COVID-19 and the Paradox of Scientific Advice -  -  #…'

In [84]:
df.head()

,created_at,id,text,cleanText
0,Tue Jun 29 12:42:36 +0000 2021,1409854813664428034,RT @MimiJ9: The government is in chaos.\n\nRam...,RT : The government is in chaos. Rampant corru...
1,Tue Jun 29 12:42:36 +0000 2021,1409854813492240388,RT @CUP_PoliSci: #FirstView from @PoPpublicsph...,RT : #FirstView from - COVID-19 and the Parad...
2,Tue Jun 29 12:42:35 +0000 2021,1409854812527632384,AstraZeneca COVID vaccine produces stronger im...,AstraZeneca COVID vaccine produces stronger im...
3,Tue Jun 29 12:42:35 +0000 2021,1409854812120834053,RT @fbhutto: Lahore based photographer Nade Al...,RT : Lahore based photographer Nade Ali is str...
4,Tue Jun 29 12:42:35 +0000 2021,1409854811307081729,RT @avarwallace: In reporting 0 positive covid...,RT : In reporting 0 positive covid tests since...


In [85]:
def getSentiment(row): 
    sentence = flair.data.Sentence(row["cleanText"])
    sentiment_model.predict(sentence)
    # probability = sentence.labels[0].score  # numerical value 0-1
    return sentence.labels[0].value  # 'POSITIVE' or 'NEGATIVE
   
def getSentimentProbability(row): 
    sentence = flair.data.Sentence(row["cleanText"])
    sentiment_model.predict(sentence)
    return sentence.labels[0].score  # numerical value 0-1
    
df["sentiment"] = df.apply(lambda row: getSentiment(row), axis = 1)
df["sentimentProbability"] = df.apply(lambda row: getSentimentProbability(row), axis = 1)

In [86]:
df.head()

,created_at,id,text,cleanText,sentiment,sentimentProbability
0,Tue Jun 29 12:42:36 +0000 2021,1409854813664428034,RT @MimiJ9: The government is in chaos.\n\nRam...,RT : The government is in chaos. Rampant corru...,NEGATIVE,0.999015
1,Tue Jun 29 12:42:36 +0000 2021,1409854813492240388,RT @CUP_PoliSci: #FirstView from @PoPpublicsph...,RT : #FirstView from - COVID-19 and the Parad...,NEGATIVE,0.814061
2,Tue Jun 29 12:42:35 +0000 2021,1409854812527632384,AstraZeneca COVID vaccine produces stronger im...,AstraZeneca COVID vaccine produces stronger im...,NEGATIVE,0.859850
3,Tue Jun 29 12:42:35 +0000 2021,1409854812120834053,RT @fbhutto: Lahore based photographer Nade Al...,RT : Lahore based photographer Nade Ali is str...,POSITIVE,0.834321
4,Tue Jun 29 12:42:35 +0000 2021,1409854811307081729,RT @avarwallace: In reporting 0 positive covid...,RT : In reporting 0 positive covid tests since...,NEGATIVE,0.984053


In [89]:
df.loc[3]["cleanText"]

'RT : Lahore based photographer Nade Ali is struggling after a fire ravaged his home. He lost his father to Covid, if you can help,…'

In [94]:


print(time_travel(now, 7))

2021-06-22T14:05:15Z


In [114]:
f = open("data.csv", "a")
f.write(df.to_csv(index=False))
f.close()